This is just a small notebook to add a "filenames" column to some important datasets: autoinhibitory AF structures, multi-domain AF structures, and single-domain AF structures.
TODO: Integrate these steps into the main pipeline.

In [27]:
import pandas as pd
import os

In [28]:
# Define some functions
def add_AF_filename(df, fp):
    # Specifically for AlphaFold files because CF have clusters

    # Get list of filenames from the file path
    filenames = os.listdir(fp)

    # Keep only the .cif files
    filenames = [f for f in filenames if f.endswith('.cif')]

    # Determine which files are in the dataframe
    df_proteins = df['uniprot'].tolist()

    # Create a dictionary of the filenames and the corresponding uniprot
    filename_dict = {}
    for filename in filenames:
        uniprot = filename.split('-')[1]
        filename_dict[uniprot] = filename

    # Create a list of the filenames in the order of the dataframe
    AF_filenames = []
    for protein in df_proteins:
        AF_filenames.append(filename_dict[protein])

    # Add the list to the dataframe
    df['filename'] = AF_filenames

    return df

Make a dataframe with filenames for the AF autoinhibitory proteins.

In [29]:
df = pd.read_csv('./project_pipeline/data/classified_files_3.tsv', sep='\t').astype('object')

# Select relevant columns
df = df[['uniprot', 'region_1', 'region_2']]

# Add the filename column
fp = './project_pipeline/data/input/Alphafold_cif/'
df = add_AF_filename(df, fp)

# Save the dataframe
df.to_csv('./project_pipeline/data/af_autoinhibited_best.tsv', sep='\t', index=False)


Add the filenames to the AF multi-domain and single-domain dataframes.

In [30]:
multi = pd.read_csv('./project_pipeline/data/multi_domain_pae.tsv', sep='\t').astype('object')
single = pd.read_csv('./project_pipeline/data/single_domain_pae.tsv', sep='\t').astype('object') # TODO add the sequence length to this file

# Add the filename column
fpm = './project_pipeline/data/input/Alphafold_multi_domain/'
fps = './project_pipeline/data/input/Alphafold_single_domain/'



# For both dataframes, we need to drop the rows with NaN values in the mean_pae column and then subset out the mean_pae column
multi = multi.dropna(subset=['mean_pae_1_1'])
multi = multi[['uniprot', 'region_1', 'region_2']]
multi = add_AF_filename(multi, fpm)

single = single.dropna(subset=['mean_pae'])
single = single[['uniprot', 'region']]
single = add_AF_filename(single, fps)

# Save the dataframe
multi.to_csv('./project_pipeline/data/af_multi_domain.tsv', sep='\t', index=False)
single.to_csv('./project_pipeline/data/af_single_domain.tsv', sep='\t', index=False)